In [116]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import glob as gb
import cv2
import seaborn as sns

In [ ]:
data=pd.read_csv('../input/dfl-bundesliga-data-shootout/train.csv')

In [117]:
data.head()

,video_id,time,event
0,1606b0e6_0,200.265822,start
1,1606b0e6_0,201.150000,challenge
2,1606b0e6_0,202.765822,end
3,1606b0e6_0,210.124111,start
4,1606b0e6_0,210.870000,challenge


In [ ]:
data.drop(['event_attributes'],inplace=True,axis=1)

In [ ]:
data.head()

In [ ]:
data_challenge=data[data['event']=='challenge']

In [ ]:
data_challenge.head()

In [ ]:
data_challenge.shape

In [ ]:
data_play=data[data['event']=='play']

In [ ]:
data_play.head()

In [ ]:
data_play.shape

In [ ]:
data_throwin=data[data['event']=='throwin']

In [ ]:
data_throwin.head()

In [ ]:
data_throwin.shape

In [ ]:
data_challenge.shape[0]

In [ ]:
sns.barplot(x=['challenge','play','throwin'],y=[data_challenge.shape[0],data_play.shape[0],data_throwin.shape[0]])

In [ ]:
!pip install moviepy

In [ ]:
os.makedirs('dataset')
os.makedirs('dataset/play')
os.makedirs('dataset/throwin')
os.makedirs('dataset/challenge')

In [ ]:
#import shutil
#shutil.rmtree("./dataset")

In [ ]:
play_path='./dataset/play'
throwin_path='./dataset/throwin'
challenge_path='./dataset/challenge'

In [ ]:
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

In [ ]:
data_throwin.shape[0]

In [ ]:
import time
def split_video_train(event_data,video_path,path_save_images,time_before_event,time_after_event,path_cutvideo,skip_frame):
    count=0
    skip_frames=0
    start_time=time.time()
    print('start time : ' , start_time)
    for i in range(event_data.shape[0]):
        index_video=event_data.iloc[i,0]
        index_time=event_data.iloc[i,1]
        ffmpeg_extract_subclip(f"{video_path}/{index_video}.mp4" , index_time - time_before_event, index_time + time_after_event , targetname="cutvideo.mp4")
        cap=cv2.VideoCapture(f'{path_cutvideo}/cutvideo.mp4')
        print('strart')
        while cap.isOpened() :
            ret,image=cap.read()
            if ret== False :
                break 
            if skip_frames % skip_frame == 0 :
                cv2.imwrite(f'{path_save_images}/{index_video}_{count}.jpg',image)
                count  = count + 1 
            skip_frames = skip_frames + 1
        cap.release()
        print('end')
    print('number of frames : ',count)
    print('end time : ', time.time() - start_time)

In [ ]:
split_video_train(event_data = data_throwin
                  ,video_path = '../input/dfl-bundesliga-data-shootout/train'
                  ,path_save_images = throwin_path
                  ,time_before_event = 0.0
                  ,time_after_event = 0.025
                  ,path_cutvideo = '.'
                  ,skip_frame = 1
                 )

In [ ]:
split_video_train(event_data = data_challenge
                  ,video_path = '../input/dfl-bundesliga-data-shootout/train'
                  ,path_save_images = challenge_path
                  ,time_before_event = 0.0
                  ,time_after_event = 0.025
                  ,path_cutvideo = '.'
                  ,skip_frame = 2
                 )

In [ ]:
split_video_train(event_data = data_play
                  ,video_path = '../input/dfl-bundesliga-data-shootout/train'
                  ,path_save_images = play_path
                  ,time_before_event = 0.0
                  ,time_after_event = 0.025
                  ,path_cutvideo = '.'
                  ,skip_frame = 12
                 )

In [ ]:
!pip install split-folders

In [ ]:
os.makedirs('./newdataset')

In [ ]:
import splitfolders

In [ ]:
splitfolders.ratio("./dataset", output="./newdataset",seed=1337, ratio=(.8, .1, .1), group_prefix=None, move=False)

In [ ]:
import tensorflow as tf
size=224
train_dir='./newdataset/train'
train_generator=tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False,
    rescale=1/255.0
).flow_from_directory(train_dir,batch_size=64,target_size=(size,size),shuffle=True)

In [ ]:
import matplotlib.pyplot as plt
classes=list(train_generator.class_indices.keys())
plt.figure(figsize=(20,20))
for X_batch, y_batch in train_generator:
    # create a grid of 3x3 images
    for i in range(0,16):
        plt.subplot(4,4,i+1)
        plt.imshow(X_batch[i])
        plt.title(classes[np.where(y_batch[i]==1)[0][0]])
    # show the plot
    plt.show()
    break

In [ ]:
sns.countplot(train_generator.classes)

In [ ]:
val_dir='./newdataset/val'
valid_generator=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255.0
).flow_from_directory(val_dir,batch_size=16,target_size=(size,size),shuffle=False)


In [ ]:

classes=list(valid_generator.class_indices.keys())
plt.figure(figsize=(20,20))
for X_batch, y_batch in valid_generator:
    # create a grid of 3x3 images
    for i in range(0,16):
        plt.subplot(4,4,i+1)
        plt.imshow(X_batch[i])
        plt.title(classes[np.where(y_batch[i]==1)[0][0]])
    # show the plot
    plt.show()
    break

In [ ]:
sns.countplot(valid_generator.classes)

In [ ]:
test_dir='./newdataset/test'
test_generator=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255.0
).flow_from_directory(test_dir,batch_size=16,target_size=(size,size),shuffle=False)


In [ ]:
classes=list(test_generator.class_indices.keys())
plt.figure(figsize=(20,20))
for X_batch, y_batch in test_generator:
    # create a grid of 3x3 images
    for i in range(0,16):
        plt.subplot(4,4,i+1)
        plt.imshow(X_batch[i])
        plt.title(classes[np.where(y_batch[i]==1)[0][0]])
    # show the plot
    plt.show()
    break

In [ ]:
sns.countplot(test_generator.classes)

In [ ]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense ,Flatten ,Dropout ,BatchNormalization 
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping ,ReduceLROnPlateau ,ModelCheckpoint

In [ ]:
#optimizer
optimizer_rmsp=RMSprop(learning_rate=0.001,rho=0.9,epsilon=1e-08,decay=0.0)

In [ ]:
#callbacks
EarlyStop=EarlyStopping(patience=5,monitor='val_loss',restore_best_weights=True)
Reduce_LR=ReduceLROnPlateau(monitor='val_loss',verbose=2,factor=0.5,min_lr=0.00001)
model_check=ModelCheckpoint('model.hdf5',monitor='val_loss',verbose=1,save_best_only=True)
callback=[EarlyStop , Reduce_LR,model_check]

In [ ]:
from tensorflow.keras.applications.xception import Xception
EfficientNetB7_model=Xception(include_top=False,input_shape=(224,224,3),weights='imagenet')
EfficientNetB7_model.trainable=False

In [ ]:
EfficientNetB7_model=Sequential([
                        EfficientNetB7_model,
                        Flatten(),
                        Dense(32,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.0001)),
                        BatchNormalization(),              
                        Dropout(0.2),
                        Dense(3,activation='softmax',kernel_initializer='glorot_normal')
                  
])


In [ ]:
EfficientNetB7_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history=EfficientNetB7_model.fit(train_generator,validation_data=valid_generator,epochs=50,
                  steps_per_epoch=len(train_generator),validation_steps=len(valid_generator),
                  callbacks=callback, verbose=1)

In [ ]:
import shutil
shutil.rmtree("./dataset")

In [ ]:
classes

In [ ]:
EfficientNetB7_model.evaluate(test_generator)

In [ ]:
predictions = EfficientNetB7_model.predict(test_generator)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_generator.classes, predictions.argmax(axis=-1)))

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_generator.classes, predictions.argmax(axis=-1))

sns.heatmap(cm,  annot=True, fmt="d" ,cmap="YlGnBu")

In [ ]:
sample_submission=pd.read_csv('../input/dfl-bundesliga-data-shootout/sample_submission.csv')

In [ ]:
sample_submission.head()

In [ ]:
sample_submission.shape

In [ ]:
plt.plot(sample_submission['time'])

In [ ]:
import glob as gb
videos=gb.glob('../input/dfl-bundesliga-data-shootout/test/*.mp4')

In [ ]:
videos

In [ ]:
classes=['challenge', 'play', 'throwin']
def predict(img):
    img=cv2.resize(img,(224,224))
    img=img/255.0
    img=np.expand_dims(img,axis=0)
    res=EfficientNetB7_model.predict(img)[0]
    res=res.argmax(axis=-1)
    return classes[res]
    

In [ ]:
events=[]
times=[]
for vid in videos :
    cap = cv2.VideoCapture(vid)
    ret,image=cap.read()
    #plt.figure(figsize=(30,20))
    #plt.imshow(image)
    res=predict(image)
    #plt.title(res)
    #plt.show()
    events.append(res)
    times.append(0)
    while cap.isOpened() :
        ret,image=cap.read()
        ret1,image1=cap.read()
        if ret1 == False :
            #plt.figure(figsize=(30,20))
            #plt.imshow(image)
            res=predict(image)
            #plt.title(res)
            #plt.show()
            events.append(res)
            times.append(30)
            break
    cap.release()
        

In [118]:
events

['challenge',
 'challenge',
 'challenge',
 'challenge',
 'challenge',
 'challenge',
 'throwin',
 'challenge',
 'challenge',
 'challenge',
 'challenge',
 'challenge',
 'throwin',
 'challenge',
 'throwin',
 'challenge',
 'challenge',
 'play',
 'challenge',
 'challenge',
 'challenge',
 'challenge',
 'challenge',
 'challenge',
 'challenge',
 'challenge',
 'throwin',
 'challenge',
 'throwin',
 'challenge',
 'play',
 'challenge',
 'challenge',
 'challenge',
 'challenge',
 'challenge',
 'challenge',
 'challenge',
 'challenge',
 'challenge',
 'play',
 'challenge',
 'challenge',
 'challenge',
 'throwin',
 'challenge',
 'challenge',
 'challenge',
 'challenge',
 'challenge',
 'challenge',
 'challenge',
 'challenge',
 'play',
 'challenge',
 'challenge',
 'challenge',
 'challenge',
 'challenge',
 'challenge',
 'challenge',
 'challenge',
 'challenge',
 'challenge']

In [119]:
sample_submission.head()

,video_id,time,event,score
0,019d5b34_0,100.00,play,0.00
1,019d5b34_0,110.50,challenge,0.03
2,019d5b34_1,90.25,throwin,0.06
3,019d5b34_1,105.00,play,0.10
4,0b1495d3_0,100.00,challenge,0.13


In [120]:
submission=sample_submission[['video_id']]

In [121]:
submission['time']=times

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [124]:
submission.head()

,video_id,time
0,019d5b34_0,0
1,019d5b34_0,30
2,019d5b34_1,0
3,019d5b34_1,30
4,0b1495d3_0,0


In [126]:
submission['score']=sample_submission['score']

In [127]:
submission.head()

,video_id,time,event,score
0,019d5b34_0,0,challenge,0.00
1,019d5b34_0,30,challenge,0.03
2,019d5b34_1,0,challenge,0.06
3,019d5b34_1,30,challenge,0.10
4,0b1495d3_0,0,challenge,0.13


In [128]:
submission.to_csv('sample_submission.csv',index=False)